In [1]:
import pandas as pd
from sqlalchemy import create_engine


## Connect to local database

In [2]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'paranormal_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

## haunted

In [3]:
csv_file = "../Resources/haunted_places.csv"
haunted_df = pd.read_csv(csv_file)
haunted_df.head(2)

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434


In [4]:
haunted_df.country.value_counts()

United States    10992
Name: country, dtype: int64

In [5]:
new_haunted_df = haunted_df.loc[haunted_df['country'] == 'United States',['state','city','longitude','latitude']]
new_haunted_df.head(2)

,state,city,longitude,latitude
0,Michigan,Ada,-85.504893,42.962106
1,Michigan,Addison,-84.381843,41.971425


In [6]:
new_haunted_df.isnull().sum()

state           0
city            3
longitude    1261
latitude     1261
dtype: int64

In [7]:
new_haunted_df.dropna(how='any', inplace=True)
new_haunted_df.isnull().sum()

state        0
city         0
longitude    0
latitude     0
dtype: int64

In [8]:
new_haunted_df.to_sql(name='haunted', con=engine, if_exists='replace', index=False)

731

## ufo

In [9]:
csv_file = "../Resources/nuforc_reports.csv"
ufo_df = pd.read_csv(csv_file, low_memory=False)
ufo_df.head(2)

,summary,country,city,state,date_time,shape,city_latitude,city_longitude,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,Long and narrow illuminated craft flying high ...,USA,Sisters,OR,2021-05-15T22:36:00,cigar,44.3156,-121.5175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,String of lights,USA,Sarasota,FL,2021-05-10T22:00:00,other,27.35480054,-82.50858279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
ufo_df.country.value_counts()

USA                           123100
Canada                          5581
United Kingdom                  3008
Australia                        861
India                            421
                               ...  
Germany (West Germany)             1
no                                 1
mid-Atlantic Ocean                 1
Atlantic Ocean/Puerto Rico         1
turkey                             1
Name: country, Length: 461, dtype: int64

In [11]:
new_ufo_df = ufo_df.loc[ufo_df['country'] == 'USA',['state','city','city_longitude','city_latitude']]
new_ufo_df["state_full"] = ""
new_ufo_df.head(2)

,state,city,city_longitude,city_latitude,state_full
0,OR,Sisters,-121.5175,44.3156,
1,FL,Sarasota,-82.50858279,27.35480054,


In [12]:
new_ufo_df.dropna(how='any', inplace=True)
new_ufo_df.isnull().sum()

state             0
city              0
city_longitude    0
city_latitude     0
state_full        0
dtype: int64

In [13]:
new_ufo_df.state.value_counts()

CA    13748
FL     6768
WA     6102
TX     5177
NY     4886
AZ     4094
PA     3823
OH     3711
IL     3698
MI     3031
OR     3018
NC     3008
CO     2751
GA     2324
IN     2309
MO     2309
MA     2293
VA     2262
NJ     2074
WI     2050
TN     1953
SC     1821
CT     1745
MN     1708
MD     1462
KY     1379
NV     1371
NM     1320
OK     1209
ID     1170
AL     1120
UT     1087
IA     1074
AR     1000
KS      997
NH      953
LA      907
ME      875
MT      846
WV      689
MS      646
NE      588
AK      502
RI      486
VT      413
HI      380
DE      324
SD      319
WY      299
ND      232
DC      126
ON        5
AB        1
BC        1
NS        1
NB        1
SK        1
MB        1
Name: state, dtype: int64

In [14]:
!pip install us

In [15]:
# https://pypi.org/project/us/
import us

In [16]:
for index, row in new_ufo_df.iterrows():
    state_abbr = row["state"]   
#     print(f"state abbr= {state_abbr}")
    state = u# hts.states.lookup(state_abbr)
    
    try:
        state_name = state.name
#         print(state_name)
        new_ufo_df.loc[index, "state_full"] = state_name
    except:
        new_ufo_df.loc[index, "state_full"] = "fail"

In [17]:
new_ufo_df = new_ufo_df.loc[new_ufo_df["state_full"]!="fail", ['state_full','city','city_longitude','city_latitude']]
new_ufo_df.head(2)

,state_full,city,city_longitude,city_latitude
0,Oregon,Sisters,-121.5175,44.3156
1,Florida,Sarasota,-82.50858279,27.35480054


In [48]:
new_ufo_df = new_ufo_df.rename(columns = {'state_full':'state','city_latitude':'latitude','city_longitude':'longitude'})
new_ufo_df

,state,city,longitude,latitude
138113,Utah,Orem,-111.668223,40.27745803
138114,Arizona,Somerton,-114.7133,32.5594


In [19]:
new_ufo_df.to_sql(name='ufos', con=engine, if_exists='replace', index=False)

437

In [20]:
len(new_ufo_df)

108437

## bigfoot

In [21]:
csv_file = "../Resources/bfro_locations.csv"
bigfoot_df = pd.read_csv(csv_file)
bigfoot_df.head(2)

,number,title,classification,timestamp,latitude,longitude
0,637,Report 637: Campers' encounter just after dark...,Class A,2000-06-16T12:00:00Z,61.5000,-142.9000
1,2917,Report 2917: Family observes large biped from car,Class A,1995-05-15T12:00:00Z,55.1872,-132.7982


In [62]:
bigfoot_df = bigfoot_df.loc[bigfoot_df['classification'] == 'Class A']

In [63]:
bigfoot_df["country"] = ""
bigfoot_df["state"] = ""
bigfoot_df["city"] = ""
bigfoot_df.head(2)

,number,title,classification,timestamp,latitude,longitude,country,state,city
0,637,Report 637: Campers' encounter just after dark...,Class A,2000-06-16T12:00:00Z,61.5000,-142.9000,,,
1,2917,Report 2917: Family observes large biped from car,Class A,1995-05-15T12:00:00Z,55.1872,-132.7982,,,


In [23]:
!pip install geopy

In [64]:
# https://www.geeksforgeeks.org/get-the-city-state-and-country-names-from-latitude-and-longitude-using-python/
from geopy.geocoders import Nominatim

In [65]:
geolocator = Nominatim(user_agent="geoapiExercises")

for index, row in bigfoot_df.iterrows():
    Latitude = str(row["latitude"])
    Longitude = str(row["longitude"])
    
    print(f"{Latitude}  {Longitude}")
    
    location = geolocator.reverse(Latitude+","+Longitude, exactly_one=True)
#     print(location)
    
    try:
        address = location.raw['address']    
        print(address)
    #     break

        country = address.get('country', '')
        bigfoot_df.loc[index, "country"] = country
        
        state = address.get('state', '')

#         try:
        bigfoot_df.loc[index, "state"] = state
#         except:
#             bigfoot_df.loc[index, "state"] = "fail"

        if len(address.get('town', '')) > 0: 
            city = address.get('town', '')
        elif len(address.get('hamlet', '')) > 0: 
            city = address.get('hamlet', '')
        elif len(address.get('village', '')) > 0: 
            city = address.get('village', '')
        elif len(address.get('city', '')) > 0: 
            city = address.get('city', '')         
        elif len(address.get('county', '')) > 0: 
            city = address.get('county', '') 
        else:
            city = "fail"

#         try:
        bigfoot_df.loc[index, "city"] = city
#         except:
#             bigfoot_df.loc[index, "city"] = "fail"
    
    except:
        bigfoot_df.loc[index, "state"] = "fail"
        bigfoot_df.loc[index, "city"] = "fail"

print("Done")    

61.5  -142.9
{'road': 'Root Glacier Trail', 'hamlet': 'Kennicott', 'county': 'Copper River', 'state': 'Alaska', 'ISO3166-2-lvl4': 'US-AK', 'country': 'United States', 'country_code': 'us'}
55.1872  -132.7982
{'road': 'Cedar Street', 'village': 'Hydaburg', 'county': 'Unorganized Borough', 'state': 'Alaska', 'ISO3166-2-lvl4': 'US-AK', 'postcode': '99922', 'country': 'United States', 'country_code': 'us'}
55.2035  -132.8202
{'road': 'Union Street', 'village': 'Hydaburg', 'county': 'Unorganized Borough', 'state': 'Alaska', 'ISO3166-2-lvl4': 'US-AK', 'postcode': '99922', 'country': 'United States', 'country_code': 'us'}
62.9375  -141.5667
{'road': 'Alaska Highway', 'county': 'Unorganized Borough', 'state': 'Alaska', 'ISO3166-2-lvl4': 'US-AK', 'country': 'United States', 'country_code': 'us'}
61.0595  -149.7853
{'road': 'East 172nd Avenue', 'county': 'Anchorage', 'state': 'Alaska', 'ISO3166-2-lvl4': 'US-AK', 'postcode': '99516', 'country': 'United States', 'country_code': 'us'}
62.77335  -14

{'house_number': '21', 'road': 'County Road 79 South', 'hamlet': 'Baker Hill', 'town': 'Bakerhill', 'county': 'Barbour County', 'state': 'Alabama', 'ISO3166-2-lvl4': 'US-AL', 'postcode': '36016', 'country': 'United States', 'country_code': 'us'}
33.8957  -87.53395
{'house_number': '701', 'road': '6th Avenue', 'village': 'Carbon Hill', 'county': 'Walker County', 'state': 'Alabama', 'ISO3166-2-lvl4': 'US-AL', 'postcode': '35549', 'country': 'United States', 'country_code': 'us'}
33.8957  -87.53395
{'house_number': '701', 'road': '6th Avenue', 'village': 'Carbon Hill', 'county': 'Walker County', 'state': 'Alabama', 'ISO3166-2-lvl4': 'US-AL', 'postcode': '35549', 'country': 'United States', 'country_code': 'us'}
34.70112  -87.77686
{'county': 'Colbert County', 'state': 'Alabama', 'ISO3166-2-lvl4': 'US-AL', 'country': 'United States', 'country_code': 'us'}
31.13586  -87.06486
{'house_number': '1900', 'road': 'Sowell Road', 'village': 'Brewton', 'county': 'Escambia County', 'state': 'Alabama

{'house_number': '103', 'road': 'East Sevier Street', 'town': 'Benton', 'county': 'Saline County', 'state': 'Arkansas', 'ISO3166-2-lvl4': 'US-AR', 'postcode': '72015', 'country': 'United States', 'country_code': 'us'}
36.37111  -91.87
{'road': 'County Road 188', 'county': 'Fulton County', 'state': 'Arkansas', 'ISO3166-2-lvl4': 'US-AR', 'postcode': '72576', 'country': 'United States', 'country_code': 'us'}
36.20611  -94.23417
{'house_number': '105', 'road': 'School Street', 'city': 'Springdale', 'county': 'Washington County', 'state': 'Arkansas', 'ISO3166-2-lvl4': 'US-AR', 'postcode': '72762', 'country': 'United States', 'country_code': 'us'}
34.70694  -94.45417
{'house_number': '3290', 'road': 'US 59;US 270', 'hamlet': 'Acorn', 'county': 'Polk County', 'state': 'Arkansas', 'ISO3166-2-lvl4': 'US-AR', 'postcode': '71953', 'country': 'United States', 'country_code': 'us'}
36.18333  -94.44694
{'house_number': '16244', 'road': 'Fairmount Road', 'hamlet': 'Martin', 'county': 'Benton County',

{'house_number': '2445', 'road': 'Batesville Pike', 'hamlet': 'Gibson', 'town': 'North Little Rock', 'county': 'Pulaski County', 'state': 'Arkansas', 'ISO3166-2-lvl4': 'US-AR', 'postcode': '72120', 'country': 'United States', 'country_code': 'us'}
34.20873  -92.57542
{'road': 'Forestry Road 9012', 'county': 'Grant County', 'state': 'Arkansas', 'ISO3166-2-lvl4': 'US-AR', 'postcode': '72084', 'country': 'United States', 'country_code': 'us'}
34.92336  -92.30519
{'road': 'Camp Joseph Robinson', 'county': 'Pulaski County', 'state': 'Arkansas', 'ISO3166-2-lvl4': 'US-AR', 'country': 'United States', 'country_code': 'us'}
35.738  -90.1355
{'county': 'Mississippi County', 'state': 'Arkansas', 'ISO3166-2-lvl4': 'US-AR', 'country': 'United States', 'country_code': 'us'}
35.186  -93.206
{'road': 'Oak Grove Road', 'county': 'Yell County', 'state': 'Arkansas', 'ISO3166-2-lvl4': 'US-AR', 'postcode': '72834', 'country': 'United States', 'country_code': 'us'}
36.042  -93.1545
{'road': 'County Road 243

{'road': 'Blakely Trail (16E07)', 'county': 'El Dorado County', 'state_district': 'CAL Fire Northern Region', 'state': 'California', 'ISO3166-2-lvl4': 'US-CA', 'country': 'United States', 'country_code': 'us'}
38.93333  -119.9833
{'house_number': '838', 'road': 'El Dorado Avenue', 'hamlet': 'Al Tahoe', 'town': 'South Lake Tahoe', 'county': 'El Dorado County', 'state': 'California', 'ISO3166-2-lvl4': 'US-CA', 'postcode': '96150', 'country': 'United States', 'country_code': 'us'}
38.86  -120.1358
{'landuse': 'Lake Aloha', 'road': 'Pacific Crest National Scenic Trail #2000', 'county': 'El Dorado County', 'state': 'California', 'ISO3166-2-lvl4': 'US-CA', 'country': 'United States', 'country_code': 'us'}
37.35861  -118.9975
{'road': 'Kaiser Pass Road', 'hamlet': 'Mono Hot Springs', 'county': 'Fresno County', 'state': 'California', 'ISO3166-2-lvl4': 'US-CA', 'country': 'United States', 'country_code': 'us'}
38.06615  -120.1877
{'house_number': '20595', 'road': 'Chief Fuller Way', 'hamlet': '

...

{'house_number': '851', 'road': 'Hilltop Avenue', 'town': 'Kent', 'county': 'King County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'postcode': '98031', 'country': 'United States', 'country_code': 'us'}
46.3105  -121.9256
{'road': 'Pinto Road', 'county': 'Skamania County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'country': 'United States', 'country_code': 'us'}
45.8911  -122.4076
{'road': 'Northeast 261st Avenue', 'county': 'Clark County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'postcode': '98601', 'country': 'United States', 'country_code': 'us'}
47.87725  -121.8943
{'house_number': '25142', 'road': '132nd Street Southeast', 'county': 'Snohomish County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'postcode': '98272', 'country': 'United States', 'country_code': 'us'}
47.0958  -122.2356
{'house_number': '17798', 'road': '146th Avenue East', 'county': 'Pierce County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'postcode': '98360', 'country': 'U

{'tourism': 'Snag Lake Campground', 'road': 'C2000', 'county': 'Pacific County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'postcode': '98638', 'country': 'United States', 'country_code': 'us'}
47.48176  -121.6548
{'road': 'Middle Fork Road', 'hamlet': 'Tanner', 'county': 'King County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'country': 'United States', 'country_code': 'us'}
46.85  -121.25
{'road': 'Richmond Mine Trail #973', 'county': 'Yakima County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'country': 'United States', 'country_code': 'us'}
46.94102  -123.7727
{'county': 'Grays Harbor County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'postcode': '98537', 'country': 'United States', 'country_code': 'us'}
47.13485  -121.9418
{'county': 'Pierce County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'country': 'United States', 'country_code': 'us'}
47.70178  -121.1859
{'road': 'Deception Creek Trail', 'county': 'King County', 'state': 'Washington', 

{'road': 'I 90', 'county': 'Kittitas County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'postcode': '98943', 'country': 'United States', 'country_code': 'us'}
47.41184  -123.3212
{'tourism': 'Brown Creek Campground', 'road': 'Forest Road 2340', 'county': 'Mason County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'country': 'United States', 'country_code': 'us'}
48.6254  -118.4262
{'road': 'State Highway 20', 'county': 'Ferry County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'country': 'United States', 'country_code': 'us'}
48.57028  -119.8345
{'county': 'Okanogan County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'country': 'United States', 'country_code': 'us'}
46.32793  -122.4349
{'road': 'Spirit Lake Highway', 'county': 'Cowlitz County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-WA', 'country': 'United States', 'country_code': 'us'}
48.03062  -117.5429
{'road': 'Oregon Road', 'county': 'Stevens County', 'state': 'Washington', 'ISO3166-2-lvl4': 'US-

{'road': 'Gavel Road', 'hamlet': 'Town of Hale', 'municipality': 'Town of Lincoln', 'county': 'Trempealeau County', 'state': 'Wisconsin', 'ISO3166-2-lvl4': 'US-WI', 'postcode': '54773', 'country': 'United States', 'country_code': 'us'}
46.10445  -91.49319
{'road': 'Jolly Fisherman Road', 'municipality': 'Town of Lenroot', 'county': 'Sawyer County', 'state': 'Wisconsin', 'ISO3166-2-lvl4': 'US-WI', 'postcode': '54843', 'country': 'United States', 'country_code': 'us'}
45.73271  -90.42001
{'municipality': 'Town of Worcester', 'county': 'Price County', 'state': 'Wisconsin', 'ISO3166-2-lvl4': 'US-WI', 'postcode': '54565', 'country': 'United States', 'country_code': 'us'}
45.02046  -87.73608
{'road': 'Harbor Road', 'municipality': 'Town of Peshtigo', 'county': 'Marinette County', 'state': 'Wisconsin', 'ISO3166-2-lvl4': 'US-WI', 'postcode': '54157', 'country': 'United States', 'country_code': 'us'}
44.75668  -92.03925
{'road': '220th Avenue', 'village': 'Town of Eau Galle', 'county': 'Dunn Co

{'road': 'Public Road 81', 'county': 'Webster County', 'state': 'West Virginia', 'ISO3166-2-lvl4': 'US-WV', 'country': 'United States', 'country_code': 'us'}
38.835  -79.3686
{'road': 'Seneca Rocks Trail', 'village': 'Seneca Rocks', 'county': 'Pendleton County', 'state': 'West Virginia', 'ISO3166-2-lvl4': 'US-WV', 'postcode': '26884', 'country': 'United States', 'country_code': 'us'}
38.8926  -79.47325
{'road': 'Hardy & Randolph Turnpike', 'county': 'Randolph County', 'state': 'West Virginia', 'ISO3166-2-lvl4': 'US-WV', 'country': 'United States', 'country_code': 'us'}
38.8409  -79.4016
{'road': 'Allegheny Drive', 'hamlet': 'Teterton', 'village': 'Seneca Rocks', 'county': 'Pendleton County', 'state': 'West Virginia', 'ISO3166-2-lvl4': 'US-WV', 'postcode': '26884', 'country': 'United States', 'country_code': 'us'}
37.50965  -80.64685
{'house_number': '2654', 'road': 'Wikel Road', 'hamlet': 'Wikel', 'county': 'Monroe County', 'state': 'West Virginia', 'ISO3166-2-lvl4': 'US-WV', 'postcode

In [66]:
new_bigfoot_df = bigfoot_df.loc[(bigfoot_df["state"]!="fail") & (bigfoot_df["country"]=="United States"), ['state','city','longitude','latitude']]
new_bigfoot_df.head(2)

,state,city,longitude,latitude
0,Alaska,Kennicott,-142.9000,61.5000
1,Alaska,Hydaburg,-132.7982,55.1872


In [67]:
new_bigfoot_df.dropna(how='any', inplace=True)
new_bigfoot_df.isnull().sum()

state        0
city         0
longitude    0
latitude     0
dtype: int64

In [68]:
new_bigfoot_df.to_sql(name='bigfoot', con=engine, if_exists='replace', index=False)

22

## Check for tables

In [69]:
print(pd.read_sql_query('select count(*) from haunted', con=engine).head())
pd.read_sql_query('select * from haunted', con=engine).head(2)

   count
0   9731


,state,city,longitude,latitude
0,Michigan,Ada,-85.504893,42.962106
1,Michigan,Addison,-84.381843,41.971425


In [70]:
print(pd.read_sql_query('select count(*) from ufos', con=engine).head())
print(pd.read_sql_query('select state, count(*) from ufos group by state order by state', con=engine).head())
print()
print(pd.read_sql_query('select distinct state, count(*) from ufos group by state', con=engine).head())
pd.read_sql_query('select * from ufos', con=engine).head(2)


    count
0  108437
        state  count
0     Alabama   1120
1      Alaska    502
2     Arizona   4094
3    Arkansas   1000
4  California  13748

        state  count
0     Alabama   1120
1      Alaska    502
2     Arizona   4094
3    Arkansas   1000
4  California  13748


,state,city,longitude,latitude
0,Oregon,Sisters,-121.5175,44.3156
1,Florida,Sarasota,-82.50858279,27.35480054


In [ ]:
states = []

In [89]:
counts_df

,state,bigfoot_sitings,reported_hauntings
0,,2.0,NaN
1,Alabama,56.0,187.0
2,Alaska,7.0,27.0
3,Arizona,22.0,144.0
4,Arkansas,68.0,98.0
5,California,104.0,1003.0
6,Colorado,61.0,150.0
7,Connecticut,5.0,176.0
8,Florida,143.0,302.0
9,Georgia,64.0,240.0


In [71]:
ufo_counts = pd.read_sql_query('select state, count(*) from ufos group by state order by state', con=engine)
haunted_counts = pd.read_sql_query('select state, count(*) from haunted group by state order by state', con=engine)
bigfoot_counts = pd.read_sql_query('select state, count(*) from bigfoot group by state order by state', con=engine)



In [72]:
bigfoot_counts_df = pd.DataFrame(bigfoot_counts).rename(columns = {'count':'bigfoot_sitings'})
haunted_counts_df = pd.DataFrame(haunted_counts).rename(columns = {'count':'reported_hauntings'})
ufo_counts_df = pd.DataFrame(ufo_counts).rename(columns = {'count':'ufo_sitings'})

In [73]:
haunted_counts_df.at[47,'state']='District of Columbia'



In [74]:
counts_df = pd.merge(bigfoot_counts_df,haunted_counts_df, on="state", how= 'outer')
all_counts_df = pd.merge(counts_df,ufo_counts_df, on="state", how= 'outer').drop([0,0])

In [75]:
print(pd.read_sql_query('select count(*) from bigfoot', con=engine).head())
pd.read_sql_query('select * from bigfoot', con=engine).head(2)

   count
0   2022


,state,city,longitude,latitude
0,Alaska,Kennicott,-142.9000,61.5000
1,Alaska,Hydaburg,-132.7982,55.1872


In [88]:
all_counts_df.fillna(0).set_index('state').T.to_json('../static/js/sitings2.json')

In [85]:
all_counts_df.fillna(0).set_index('state').to_json('../static/js/sitings.json')

In [ ]:

ufo_dict = {
    "states": [
{
"name": 'California',
"sitings": 13748,
},
{
"name": 'Florida',
"sitings": 6768,
},
{
"name": 'Washington',
"sitings": 6102,
},
{
"name": 'Texas',
"sitings": 5177,
},
{
"name": 'New York',
"sitings": 4886,
},
{
"name": 'Arizona',
"sitings": 4094,
},        
{
"name": 'Pennsylvania',
"sitings": 3823,
},
{
"name": 'Ohio',
"sitings": 3711,
},
{
"name": 'Illinois',
"sitings": 3698,
},
{
"name": 'Michigan',
"sitings": 3031,
},
{
"name": 'Oregon',
"sitings": 3018,
},
{
"name": 'North Carolina',
"sitings": 3008,
},
{
"name": 'Colorado',
"sitings": 2751,
},
{
"name": 'Georgia',
"sitings": 2324,
},
{
"name": 'Indiana',
"sitings": 2309,
},
{
"name": 'Missouri',
"sitings": 2309,
},
{
"name": 'Massachusetts',
"sitings": 2293,
},
{
"name": 'Virginia',
"sitings": 2262,
},
{
"name": 'New Jersey',
"sitings": 2293,
},
{
"name": 'Wisconsin',
"sitings": 2050,
},
{
"name": 'Tennessee',
"sitings": 1953,
},
{
"name": 'South Carolina',
"sitings": 1821,
},
{
"name": 'Connecticut',
"sitings": 1745,
},
{
"name": 'Minnesota',
"sitings": 1708,
},
{
"name": 'Maryland',
"sitings": 1462,
},
{
"name": 'Kentucky',
"sitings": 1379,
},     
{
"name": 'Nevada',
"sitings": 1371,
},   
{
"name": 'New Mexico',
"sitings": 1371,
},
{
"name":'Oklahoma',
"sitings": 1209,
},
{
"name":'Idaho',
"sitings": 1170,
},
{
"name":'Alabama',
"sitings": 1120,
},    
{
"name":'Utah',
"sitings": 1087,
},
{
"name":'Iowa',
"sitings": 1074,
},
{
"name":'Arkansas',
"sitings": 1000,
},
{
"name":'Kansas',
"sitings": 997,
},
{
"name":'New Hampshire',
"sitings": 953,
},
{
"name":'Louisiana',
"sitings": 907,
},       
{
"name":'Maine',
"sitings": 875,
},
{
"name":'Montana',
"sitings": 846,
},
{
"name":'West Virginia',
"sitings": 689,
},
{
"name":'Mississippi',
"sitings": 846,
},
    ],
}




In [ ]:
ufos_by_state